# Fastembed - Embed and Store Images with Labels in Qdrant

In [201]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [202]:
#!pip install qdrant-client fastembed

In [203]:
#!python3.12 -m ipykernel install --user --name python3.12 --display-name "Python 3.12"


In [204]:
#!python3.12 -m pip install ipykernel

In [205]:
import sys
print(sys.version)


3.12.3 (main, Jun  3 2024, 12:49:48) [GCC 11.4.0]


In [206]:
!tree -L 2

.
├── env
│   ├── bin
│   ├── lib
│   └── pyvenv.cfg
├── Fast_Embed_Images.ipynb
├── fmb
│   ├── bin
│   ├── lib
│   └── pyvenv.cfg
└── images_dataset
    ├── human
    └── not_human

9 directories, 3 files


In [207]:
from qdrant_client import QdrantClient, models
from fastembed import ImageEmbedding
from qdrant_client.models import PointStruct, VectorParams, Distance
import os
from PIL import Image

In [208]:

# Initialize lists to store image paths and labels
image_paths = []
labels = []

# Directories for human and not human images
root_dir = 'images_dataset'

# Collect image paths and labels
for label_dir in ['human', 'not_human']:
    label_path = os.path.join(root_dir, label_dir)
    for subdir, _, files in os.walk(label_path):
        for file in files:
            if file.lower().endswith(".jpg"):
                image_paths.append(os.path.join(subdir, file))
                labels.append(label_dir)

# Initialize the ImageEmbedding model
model = ImageEmbedding(model_name="Qdrant/clip-ViT-B-32-vision")

# Generate embeddings
embeddings = list(model.embed(image_paths))


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [209]:
#embeddings

In [210]:
# Create Qdrant collection
collection_name = "image_classification"
client.delete_collection(collection_name)
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE)
)


True

In [211]:
# Insert points into Qdrant
points = [
    PointStruct(id=idx, vector=embedding, payload={"path": image_paths[idx], "label": labels[idx]})
    for idx, embedding in enumerate(embeddings)
]
client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [212]:
points

[PointStruct(id=0, vector=[0.025790007784962654, 0.010247795842587948, -0.04418972507119179, -0.023581041023135185, -0.01699051260948181, -0.05749715864658356, 0.011033975519239902, 0.03789930418133736, -0.020612049847841263, 0.016881778836250305, -0.005460378248244524, -0.0012602326460182667, -0.0304368045181036, 0.0029165574815124273, 0.04193374514579773, 0.013535808771848679, 0.05706631392240524, -0.003609202103689313, 0.0289594829082489, 0.0038476851768791676, 0.010939805768430233, 0.006893184967339039, -0.058453213423490524, 0.01375805214047432, -0.015486062504351139, 0.004969937261193991, 0.012486657127737999, 0.016431547701358795, -0.0036829598248004913, 0.0261636171489954, -0.028397148475050926, -0.01370895467698574, -0.030551724135875702, -0.06383541971445084, -0.04570845514535904, 0.03814396634697914, -0.0112808458507061, 0.04596693813800812, -0.02036704309284687, -0.04544953629374504, 0.03583359345793724, 0.0568995364010334, -0.03763928636908531, -0.01537276990711689, 0.0125